In [1]:
import redis
from imdb import IMDb

In [46]:
#CONNECT TO REDIS
r = redis.StrictRedis(host='localhost', port=6379, db=0)
r2 = redis.StrictRedis(host='localhost', port=6379, db=1)
r3 = redis.StrictRedis(host='localhost', port=6379, db=2)
pipe3 = r3.pipeline()
pipe2 = r2.pipeline()
pipe = r.pipeline()

In [3]:
#GET BOTH IMDBPIE AND IMDB ACCESS 
from imdbpie import Imdb
imdb1 = Imdb()
imdb1 = Imdb(anonymize=True) # to proxy requests

# Creating an instance with caching enabled
# Note that the cached responses expire every 2 hours or so.
# The API response itself dictates the expiry time)
imdb1 = Imdb(cache=True)

In [4]:
#GET THE TOP 250 MOVIES, and put id as key and movie name as value
top250 = imdb1.top_250()
dict_top250 = {}
for i in range(len(top250)):
    dict_top250[((top250[i]['tconst'].encode('utf-8'))[2:len(top250[i]['tconst'].encode('utf-8'))])] = (top250[i]['title'].encode('utf-8'))

In [5]:
#For each movieid and title, get top 10 cast list
cast_dict = {}

for movId, title in dict_top250.iteritems():
    cast_list = []
    imdb2 = IMDb()
    my = imdb2.get_movie(movId)
    for castMember in my['cast'][0:10]:
            cast_list.append(castMember.getID());
    cast_dict[movId] = cast_list
print cast_dict

{'0061512': ['0000056', '0001421', '0134201', '0031398', '0238105', '0001510', '0886888', '0416378', '0940971', '0039435'], '0482571': ['0413168', '0000288', '0000323', '0005305', '0356017', '0424060', '2195817', '0000309', '0785227', '0204425'], '0018455': ['0639563', '0310980', '0515272', '0743017', '0531759', '0802684', '0936079', '0396768', '0092665', '0201576'], '0114709': ['0000158', '0000741', '0725543', '0001815', '0001728', '0001652', '0001633', '0606658', '0902184', '0582418'], '0083987': ['0001426', '0000298', '0002081', '0000024', '0002145', '0590055', '0000640', '0153182', '0297538', '0356847'], '2084970': ['1212722', '0461136', '0328828', '1239499', '1395602', '0063914', '0001097', '0835016', '3485153', '0329259'], '0071562': ['0000199', '0000380', '0000473', '0000134', '0001030', '0001735', '0833448', '0311155', '0819525', '0109175'], '0045152': ['0000037', '0640307', '0001666', '0353405', '0593612', '0001998', '0288830', '0001549', '0011625', '0017021'], '1049413': ['00

In [7]:
def test(movies):
    """input: list of 250 movies
        output: hashtable"""
    act_assoc_map = {}
    for movie in movies:
        l_act = movies.get(movie)
        for act in l_act:
            for act_id in range(len(l_act)):
                if act not in act_assoc_map.keys():
                    if l_act[act_id] != act:
                        act_assoc_map[act] = [l_act[act_id]]
                else:
                    if l_act[act_id] != act:
                        act_assoc_map[act].append(l_act[act_id])
    for actor in act_assoc_map.keys():
        act_assoc_map[actor] = list(set(act_assoc_map[actor]))
    return act_assoc_map

hashtable = test(cast_dict)

#Push hashtable items into redis db0
for key, value in hashtable.iteritems():
    print str(value)
    pipe.execute();
    for i in value:
        pipe.lpush(key, i)

['0416694', '0285495', '0717372', '0000093', '0311563', '0005458', '0001125', '0001199', '0005068']
['0281313', '0402032', '0000450', '0577513', '1814260', '0001057', '0315417', '0051509', '2512626']
['0088127', '0000959', '0202002', '1617514', '0206257', '0161980', '0004695', '0001125', '0209962']
['0336475', '0706368', '0675490', '0831889', '0000348', '0000493', '0000054', '0002285', '0790064']
['0001695', '0219666', '0060904', '0000752', '0807580', '0000059', '0178259', '0178066', '0115558']
['0002034', '0335788', '0001017', '0001673', '0001012', '0842137', '0002011', '0275835', '0000842', '0925713', '0550855', '0912001', '0834754', '0001430', '0938231', '0003225', '0001181', '0801193', '0083081', '0832375', '0000314', '0275213', '0703600', '0943978', '0100758', '0863193', '0702798']
['0335788', '0000992', '0000075', '0002121', '0000638', '0087404', '0000017', '0001703', '0355095', '0001050', '0533247', '0604656', '0909930', '0481618', '0001961', '0000044', '0459252', '0001847']
['0

In [52]:
#Python implementation of bfs
def bfs(graph, start, end):
    # maintain a queue of paths
    queue = []
    # push the first path into the queue
    queue.append([start])
    while queue:
        # get the first path from the queue
        path = queue.pop(0)
        # get the last node from the path
        node = path[-1]
        # path found
        if node == end:
            return path
        # enumerate all adjacent nodes, construct a new path and push it into the queue
        for adjacent in graph.get(node, []):
            new_path = list(path)
            new_path.append(adjacent)
            queue.append(new_path)

#Example execution of bfs
list_relation = bfs(hashtable, '0000138', '0000012')

In [53]:
#Print the people in the list's names
for i in list_relation:
    full_person = imdb2.get_person(i)
    print full_person

Leonardo DiCaprio
Martin Sheen
Robert Duvall
Wesley Addy
Bette Davis


In [23]:
# GET THE ACTOR ID AND NAME INTO A DICTIONARY FOR LOOKUP
actor_id_dict = {}
count = 0
for i in hashtable.keys():
    full_person = imdb2.get_person(i)
    actor_id_dict[i] = full_person["name"]
    count += 1
    print count

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


2015-12-04 16:49:26,718 CRITICAL [imdbpy] /home/cynchen/anaconda/lib/python2.7/site-packages/IMDbPY-5.0-py2.7-linux-x86_64.egg/imdb/_exceptions.py:35: IMDbDataAccessError exception raised; args: ({'exception type': 'IOError', 'url': 'http://akas.imdb.com/name/nm2438307/maindetails', 'errcode': 'socket error', 'proxy': '', 'original exception': IOError('socket error', error(104, 'Connection reset by peer')), 'errmsg': '[Errno 104] Connection reset by peer'},); kwds: {}
Traceback (most recent call last):
  File "/home/cynchen/anaconda/lib/python2.7/site-packages/IMDbPY-5.0-py2.7-linux-x86_64.egg/imdb/parser/http/__init__.py", line 202, in retrieve_unicode
    uopener = self.open(url)
  File "/home/cynchen/anaconda/lib/python2.7/urllib.py", line 213, in open
    return getattr(self, name)(url)
  File "/home/cynchen/anaconda/lib/python2.7/urllib.py", line 351, in open_http
    errcode, errmsg, headers = h.getreply()
  File "/home/cynchen/anaconda/lib/python2.7/httplib.py", line 1207, in ge


906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003

2015-12-04 16:52:25,297 CRITICAL [imdbpy] /home/cynchen/anaconda/lib/python2.7/site-packages/IMDbPY-5.0-py2.7-linux-x86_64.egg/imdb/_exceptions.py:35: IMDbDataAccessError exception raised; args: ({'exception type': 'IOError', 'url': 'http://akas.imdb.com/name/nm0919798/maindetails', 'errcode': None, 'proxy': '', 'original exception': timeout('timed out',), 'errmsg': 'None'},); kwds: {}
Traceback (most recent call last):
  File "/home/cynchen/anaconda/lib/python2.7/site-packages/IMDbPY-5.0-py2.7-linux-x86_64.egg/imdb/parser/http/__init__.py", line 206, in retrieve_unicode
    content = uopener.read(**kwds)
  File "/home/cynchen/anaconda/lib/python2.7/socket.py", line 355, in read
    data = self._sock.recv(rbufsize)
timeout: timed out
CRITICAL:imdbpy:IMDbDataAccessError exception raised; args: ({'exception type': 'IOError', 'url': 'http://akas.imdb.com/name/nm0919798/maindetails', 'errcode': None, 'proxy': '', 'original exception': timeout('timed out',), 'errmsg': 'None'},); kwds: {}
Tr


1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201
1202
1203

In [52]:
#Put in the actor id and actor name into dictionary
new_actors_ids = {}
for key,value in actor_id_dict.iteritems():
    try:
        new_value = str(value)
        new_new_value = new_value.replace(" ", "")
        new_actors_ids[key] = new_new_value
    except:
        new_value = value
        new_new_value = new_value.replace(" ", "")
        new_actors_ids[key] = new_new_value
new_actors_ids

{'0784884': 'RadeSerbedzija',
 '0287054': 'JulieForsyth',
 '1226817': 'DevonAoki',
 '0113873': 'JoeE.Brown',
 '0000021': 'JoanFontaine',
 '0000020': 'HenryFonda',
 '0000023': 'JudyGarland',
 '0665708': 'AngelaPaton',
 '0000024': 'JohnGielgud',
 '0000027': 'AlecGuinness',
 '0000026': 'CaryGrant',
 '1554735': u'CharlyH\xfcbner',
 '1027683': 'LeonaAderias',
 '1027682': 'FrankAderias',
 '0550855': 'E.G.Marshall',
 '0147566': 'PietroCeccarelli',
 '0602071': 'MargoMoorer',
 '0876300': 'UlrichTukur',
 '0496819': 'AnnaLee',
 '1727304': 'DomhnallGleeson',
 '0848667': 'AkimTamiroff',
 '0927711': 'NaimaWifstrand',
 '0728812': 'ThelmaRitter',
 '0156432': "TungCho'Joe'Cheung",
 '0388517': 'HeinzHoenig',
 '0352278': 'MatthiasHabich',
 '0880521': 'LivUllmann',
 '0005476': 'HilarySwank',
 '0647970': 'NancyOlson',
 '0021600': 'BruceAllpress',
 '0785286': 'JacquesSernas',
 '0000022': 'ClarkGable',
 '0587060': 'BenMiles',
 '0477981': u'DianaK\xf6rner',
 '0112400': 'JedBrophy',
 '0001088': 'PeterCushing',

In [56]:
import pickle

#Make the pickle file with dicionary of names and id's
with open('actors_ids2.pickle', 'wb') as handle:
  pickle.dump(actor_id_dict, handle)

#Open th pickle file
with open('actors_ids2.pickle', 'rb') as handle:
  new_actor_ids = pickle.load(handle)

print new_actor_ids

{'0784884': 'Rade Serbedzija', '0287054': 'Julie Forsyth', '1226817': 'Devon Aoki', '0803310': 'Ping Lam Siu', '0000021': 'Joan Fontaine', '0000020': 'Henry Fonda', '0000023': 'Judy Garland', '0665708': 'Angela Paton', '0000024': 'John Gielgud', '0000027': 'Alec Guinness', '0000026': 'Cary Grant', '1554735': u'Charly H\xfcbner', '1027683': 'Leona Aderias', '1027682': 'Frank Aderias', '0550855': 'E.G. Marshall', '0147566': 'Pietro Ceccarelli', '0602071': 'Margo Moorer', '0876300': 'Ulrich Tukur', '0496819': 'Anna Lee', '1727304': 'Domhnall Gleeson', '0910607': 'Christoph Waltz', '0848667': 'Akim Tamiroff', '0927711': 'Naima Wifstrand', '0728812': 'Thelma Ritter', '0156432': "Tung Cho 'Joe' Cheung", '0388517': 'Heinz Hoenig', '0352278': 'Matthias Habich', '0880521': 'Liv Ullmann', '0005476': 'Hilary Swank', '0647970': 'Nancy Olson', '1910255': 'Dylan Minnette', '0785286': 'Jacques Sernas', '0000022': 'Clark Gable', '0587060': 'Ben Miles', '0477981': u'Diana K\xf6rner', '0112400': 'Jed Br

In [59]:
#Put into db1: ids, names
for key, value in new_actor_ids.iteritems():
    pipe2.execute();
    pipe2.set(key, value)

In [ ]:
#Put into db2: names, ids
for key, value in new_actors_ids.iteritems():
    pipe3.execute();
    pipe3.set(value, key)